# Read in a csv file from TEPCat
# and allow some simple comparisons.

In [1]:
% matplotlib inline
from matplotlib import rcParams
#rcParams['figure.figsize'] = 8, 6
from matplotlib.font_manager import FontProperties
import scipy.integrate as integrate
import csv
import math
import numpy as np
import pylab as p
#import my_fns

# For magnitude lookups
from astroquery.irsa import Irsa
import astropy.units as u
import astropy.constants as cs

# For TESS
import tess_stars2px as tesspt

from exo_fns import *

Loading BokehJS ...

Ehrenreich+14 staring mode GJ3470,
follow up done in spatial scanning by Tsiaras(?)

### Noise Scalings

In [46]:
noise_scalings = {'WFC3_G141': 
                      {'Transit': {'System':'GJ_1214', 'Noise':70e-6 , 'Ref':'Kreidberg+14'},
                       'Eclipse': {'System':'GJ_1214', 'Noise':70e-6 , 'Ref':'Kreidberg+14'}},
                  'WFC3_G102': 
                      {'Transit': {'System':'WASP-012', 'Noise':164e-6 , 'Ref':'?'},
                       'Eclipse': {'System':'WASP_012', 'Noise':164e-6 , 'Ref':'?'}},
                  'STIS': 
                      {'Transit': {'System': 'HD_209458', 'Noise':52e-6, 'Ref':'Charbonneau+02'},
                       'Eclipse': {'System': 'HD_209458', 'Noise':110e-6, 'Ref':'Brown+01'}}
                  }

# Transmission

## Transit depths

In [39]:
def get_transit_scaling(instrument, scale_band, noise_scalings, plot=False):

    od = get_tepcat_planets()

    od['g'] = calc_surface_g(od['M_b']*cs.M_jup.value, od['R_b']*cs.R_jup.value)
    od['TEQ'] = calc_eq_temp(od['a(AU)']*cs.au.value, od['Teff'], od['R_A']*cs.R_sun.value, A=0)
    od['SH'] = calc_scale_h(od['TEQ'], od['g'])
    od['ATMDEPTH'] = calc_transit_depth(od['SH'], od['R_b']*cs.R_jup.value, od['R_A']*cs.R_sun.value)
    
    
    scale = noise_scalings[instrument]['Transit']
    scale_ind = list(od['System']).index(scale['System'])
    scale_N = scale['Noise']
    scale_mag = od[scale_band][scale_ind]
    
    
    od['TSNR'] = (10**(-(od[scale_band]-scale_mag)/2.5/2.))*od['ATMDEPTH'] / scale_N
    
    od['ATMDEPTH'] *= 1e6 # for visualization
    table_name = './tables/{}/transit_snr_{}.csv'.format(instrument, scale_band[0])
    csv_arrays(table_name, od, 
       keys=['System', scale_band, 'ATMDEPTH', 'TSNR', 'Period', 'M_b', 'g', 'TEQ'],
       names=['Name', scale_band, 'AtmDepth','SNR', 'Period', 'Mass', 'logg', 'Eq Temp'],
       units = ['-', '-', 'ppm', '-', 'days', 'M_jup', '-', 'K'],
       formats=['{}','{:.2f}', '{:.1f}', '{:.3f}', '{:.2f}', '{:.2f}', '{:.2f}', '{:.0f}'],
       sort='TSNR', reverse=True, finite=True)
    if plot:
        afile = open(table_name, 'r+')
        csvReader1 = csv.reader(afile)
        lines = [ csvReader1.next() for _ in range(22) ]
        for line in lines:
            for val in line:
                print '{:<12s}'.format(val),
            print 
    
    ymax = np.max(od['TSNR'][np.isfinite(od['TSNR'])])
    plot_html(od, xname='M_b', yname='TSNR', x_range=(-0.1,3), y_range=(0,ymax), 
        title='Transmission, {}-band with {}'.format(scale_band[0], instrument), 
        fname='./tables/{}/transit_snr_{}.html'.format(instrument, scale_band[0]), plot=plot)
    od['ATMDEPTH'] /= 1e6
    
    return od

#### Example

In [40]:
# WFC3 G141
od = get_transit_scaling('WFC3_G141', 'Kmag', noise_scalings, plot=True)

Name         Kmag         AtmDepth     SNR          Period       Mass         logg         Eq Temp     
-            -            ppm          -            days         M_jup        -            K           
HD_189733    5.54         272.3        17.299       2.22         1.15         21.52        1417        
HD_209458    6.31         387.9        17.282       3.52         0.71         9.29         1735        
WASP-069     7.46         643.6        16.886       3.87         0.26         5.77         1142        
WASP-107     8.64         1102.5       16.799       5.72         0.12         3.45         876         
KELT-11      6.12         330.2        16.057       4.74         0.17         2.33         2026        
WASP-127     8.64         1033.4       15.746       4.18         0.18         2.38         1705        
GJ_1214      8.78         992.4        14.177       1.58         0.02         7.57         679         
WASP-094     8.87         675.3        9.256        3.95        

In [41]:
# STIS (scaled to Charbennau+02)
od = get_transit_scaling('STIS', 'Vmag', noise_scalings, plot=True)

Name         Vmag         AtmDepth     SNR          Period       Mass         logg         Eq Temp     
-            -            ppm          -            days         M_jup        -            K           
HD_209458    7.65         387.9        7.459        3.52         0.71         9.29         1735        
WASP-127     10.16        1033.4       6.255        4.18         0.18         2.38         1705        
KELT-11      8.03         330.2        5.330        4.74         0.17         2.33         2026        
HD_189733    7.68         272.3        5.165        2.22         1.15         21.52        1417        
WASP-069     9.87         643.6        4.453        3.87         0.26         5.77         1142        
WASP-094     10.06        675.3        4.281        3.95         0.45         3.79         1920        
WASP-033     8.18         256.6        3.866        1.22         2.16         18.99        3252        
WASP-076     9.53         473.5        3.831        1.81        

# Emission

In [42]:
def get_emission_scaling(instrument, scale_band, noise_scalings, plot=False, Ag=0, reflection=False):
        
    od = get_tepcat_planets()

    od['g'] = calc_surface_g(od['M_b']*cs.M_jup.value, od['R_b']*cs.R_jup.value)
    od['Ag'] = np.ones(len(od['System']))*Ag
    od['TEQ'] = calc_eq_temp(od['a(AU)']*cs.au.value, od['Teff'], od['R_A']*cs.R_sun.value, A=Ag)

    band = scale_band[0] # letter V or K
    od['EDE'] = eclipse_depth(od['TEQ'], od['Teff'], od['R_b']*cs.R_jup.value, od['R_A']*cs.R_sun.value, band=band)
    od['EDR'] = reflected_depth(od['Teff'], od['R_b']*cs.R_jup.value, od['R_A']*cs.R_sun.value, od['a(AU)']*cs.au.value, band=band, A=Ag)
    od['ED'] = od['EDE'] + od['EDR']
        
    scale = noise_scalings[instrument]['Eclipse']
    scale_ind = list(od['System']).index(scale['System'])
    scale_N = scale['Noise']
    scale_mag = od[scale_band][scale_ind]
    
    
    od['EESNR'] =  (10**(-(od[scale_band]-scale_mag)/2.5/2.))*od['EDE'] / scale_N
    od['ERSNR'] =  (10**(-(od[scale_band]-scale_mag)/2.5/2.))*od['EDR'] / scale_N
    od['ESNR'] =   (10**(-(od[scale_band]-scale_mag)/2.5/2.))*(od['ED']) / scale_N
    od['E_N'] =  scale_N / (10**(-(od[scale_band]-scale_mag)/2.5/2.))*1e6
        
    od['EDE'] *= 1e6; od['EDR'] *= 1e6; od['ED'] *= 1e6
    table_name = './tables/{}/emission_snr_{}_A-{}.csv'.format(instrument, scale_band[0], Ag)
    csv_arrays(table_name, od, 
       keys=['System', scale_band, 'EDE', 'EDR', 'E_N', 'Ag', 'TEQ', 'M_b'],
       names=['Name', scale_band, 'Depth (Em)', 'Depth (Rf)', 'Noise', 'Albedo', 'Eq Temp', 'Mass'],
       units = ['-', '-', 'ppm', 'ppm', 'ppm', '-', 'K', 'M_jup'],
       formats=['{}','{:.2f}', '{:.1f}', '{:.1f}', '{:.1f}', '{:.2f}', '{:.0f}', '{:.2f}'],
       sort='ESNR', reverse=True, finite=True)
    if plot:
        afile = open(table_name, 'r+')
        csvReader1 = csv.reader(afile)
        lines = [ csvReader1.next() for _ in range(22) ]
        for line in lines:
            for val in line:
                print '{:<10s}'.format(val),
            print 
    
    ymax = np.max(od['ED'][np.isfinite(od['ED'])])
    plot_html(od, xname='M_b', yname='ED', x_range=(-0.1,13), y_range=(0,ymax), 
        title='Eclipse depth, {}-band with {}, Ag={}'.format(scale_band[0], instrument, Ag), 
        fname='./tables/{}/eclipse_snr_{}_A-{}.html'.format(instrument, scale_band[0], Ag), plot=plot)

In [35]:
# WFC3 G141 emission
od = get_emission_scaling('WFC3_G141', 'Kmag', noise_scalings, plot=True, Ag=0.)

Name       Kmag       Depth (Em) Depth (Rf) Noise      Albedo     Eq Temp    Mass      
-          -          ppm        ppm        ppm        -          K          M_jup     
WASP-033   7.47       2795.6     0.0        38.3       0.00       3252       2.16      
KELT-09    7.48       2090.9     0.0        38.5       0.00       4817       2.88      
HD_189733  5.54       613.9      0.0        15.7       0.00       1417       1.15      
WASP-018   8.13       1924.5     0.0        51.9       0.00       2868       10.52     
KELT-20    7.42       1356.4     0.0        37.4       0.00       2687       0.00      
WASP-121   9.37       3178.8     0.0        91.9       0.00       2804       1.18      
WASP-076   8.24       1858.6     0.0        54.6       0.00       2595       0.92      
WASP-189   6.06       647.0      0.0        20.0       0.00       3141       2.13      
HD_209458  6.31       639.4      0.0        22.4       0.00       1735       0.71      
WASP-077   8.40       1491.8    

In [36]:
# STIS reflection
od = get_emission_scaling('STIS', 'Vmag', noise_scalings, plot=True, Ag=.9)

Name       Vmag       Depth (Em) Depth (Rf) Noise      Albedo     Eq Temp    Mass      
-          -          ppm        ppm        ppm        -          K          M_jup     
WASP-033   8.18       0.3        215.9      66.4       0.90       1829       2.16      
KELT-09    7.56       5.4        153.3      49.9       0.90       2709       2.88      
WASP-018   9.27       0.1        176.4      109.6      0.90       1613       10.52     
WASP-121   10.52      0.1        276.2      195.0      0.90       1577       1.18      
HD_189733  7.68       0.0        69.0       52.7       0.90       797        1.15      
WASP-076   9.53       0.0        158.0      123.6      0.90       1459       0.92      
WASP-189   6.64       0.0        39.3       32.7       0.90       1766       2.13      
KELT-20    7.59       0.0        52.7       50.6       0.90       1511       0.00      
WASP-012   11.69      0.3        338.8      334.2      0.90       1713       1.47      
MASCARA-1  8.27       0.0       

### TESS planets in order

In [3]:
od = get_tepcat_planets(TESS=True)

first_sectors = []
for planet, sec in zip(od['System'], od['TESS_sectors']):
    if len(sec) > 0:
        first_sectors.append(13-min(sec))
    else:
        first_sectors.append(None)

_, _, i_TESSbest = zip(*sorted(zip(od['TESS_days'], first_sectors, np.arange(len(od['System']))), reverse=True))

with open('TESS_Sectors.txt', 'w') as g:
    g.write('System\tDays\tSectors\n')
    for i in i_TESSbest:
            if od['Dec'][i] < 0: 
                if od['TESS_days'][i] > 0:
                    line = '{:10s}\t{}\t{}\n'.format(od['System'][i], od['TESS_days'][i], od['TESS_sectors'][i])
                    g.write(line)
                    print line[:-1]

WASP-069  	351	[ 1  2  3  4  5  6  7  8  9 10 11 12 13]
WASP-031  	297	[ 1  3  4  5  6  7  8  9 10 11 13]
pi_Men    	216	[ 1  4  5  7  8 11 12 13]
HATS-42   	54	[1 2]
HATS-30   	54	[1 2]
WASP-074  	54	[2 3]
WASP-066  	54	[2 3]
WASP-042  	54	[3 4]
WASP-117  	54	[4 5]
WASP-087  	54	[4 5]
HATS-52   	54	[4 5]
HATS-40   	54	[5 6]
HATS-39   	54	[5 6]
WASP-126  	54	[6 7]
TRAPPIST-1d	54	[6 7]
HATS-57   	54	[6 7]
HATS-44   	54	[6 7]
HATS-41   	54	[6 7]
HATS-35   	54	[6 7]
HATS-34   	54	[6 7]
WASP-089  	54	[7 8]
HATS-55   	54	[8 9]
HATS-22   	54	[8 9]
EPIC_249624646c	54	[ 9 10]
HATS-23   	54	[10 11]
HATS-13   	54	[10 11]
WASP-122  	27	[1]
WASP-108  	27	[1]
WASP-107  	27	[1]
WASP-099  	27	[1]
WASP-091  	27	[1]
WASP-080  	27	[1]
WASP-064  	27	[1]
WASP-063  	27	[1]
WASP-043  	27	[1]
OGLE-TR-113	27	[1]
HATS-10   	27	[1]
HATS-09   	27	[1]
WASP-129  	27	[2]
WASP-123  	27	[2]
WASP-109  	27	[2]
OGLE-TR-132	27	[2]
OGLE-TR-111	27	[2]
OGLE-TR-056	27	[2]
OGLE-TR-010	27	[2]
HATS-59   	27	[2]
HATS-28   	27	[2